<a href="https://colab.research.google.com/github/EliasNoorzad/XAI_Autonomous-Driving/blob/main/evaluation/03_tritask_cbam_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [4]:
!pip -q install ultralytics==8.4.6


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.4 MB/s eta 0:00:00


In [2]:
!cp /content/drive/MyDrive/XAI_Project/BDD100K_640.zip /content/

In [3]:
!unzip -q /content/BDD100K_640.zip -d /content/BDD100K_640

In [13]:
!cp /content/drive/MyDrive/XAI_Project/daynight_labels.csv /content/

In [5]:
yaml = """\
path: /content/BDD100K_640/yolo_640
train: train/images
val: val/images
test: test/images

nc: 5
names: [car, truck, bus, person, bike]
"""

with open("/content/BDD100K_640/yolo_640/dataset_640.yaml", "w") as f:
    f.write(yaml)

In [6]:
import csv
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset


class BDDDetDrivableDataset(Dataset):
    """
    For preprocessed 640 dataset (yolo_640 + drivable_masks_640):
      images: <yolo_root>/<split>/images/<stem>.jpg
      labels: <yolo_root>/<split>/labels/<stem>.txt   (already corrected for 640)
      masks : <mask_root>/<split>/<stem>.png          (already 640)

    Returns:
      img   : FloatTensor [3, H, W] in [0,1]
      labels: FloatTensor [N, 5] where each row is [cls, x, y, w, h] (YOLO normalized)
      mask  : FloatTensor [1, H, W] with values 0/1
      dn    : LongTensor scalar (0=day, 1=night)  <-- always valid (unlabeled images are filtered out)
    """
    def __init__(
        self,
        yolo_root: str,
        mask_root: str,
        split: str,
        imgsz: int = 640,
        dn_csv_path: str | None = None,
    ):
        self.yolo_root = Path(yolo_root)
        self.mask_root = Path(mask_root)
        self.split = split
        self.imgsz = int(imgsz)

        self.img_dir = self.yolo_root / split / "images"
        self.lbl_dir = self.yolo_root / split / "labels"
        self.msk_dir = self.mask_root / split

        if not self.img_dir.is_dir():
            raise FileNotFoundError(f"Missing images dir: {self.img_dir}")
        if not self.lbl_dir.is_dir():
            raise FileNotFoundError(f"Missing labels dir: {self.lbl_dir}")
        if not self.msk_dir.is_dir():
            raise FileNotFoundError(f"Missing masks dir:  {self.msk_dir}")

        exts = {".jpg", ".jpeg", ".png"}
        self.img_paths = sorted([p for p in self.img_dir.iterdir() if p.suffix.lower() in exts])
        if len(self.img_paths) == 0:
            raise FileNotFoundError(f"No images found in: {self.img_dir}")

        # ---------- day/night mapping from CSV (REQUIRED for Stage B) ----------
        if dn_csv_path is None:
            raise RuntimeError("dn_csv_path is required for this dataset (day/night head training).")

        dn_csv_path = Path(dn_csv_path)
        if not dn_csv_path.exists():
            raise FileNotFoundError(f"Missing day/night CSV: {dn_csv_path}")

        dn_map = {}
        with open(dn_csv_path, "r", newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            required = {"split", "image_id", "label"}
            if not required.issubset(set(reader.fieldnames or [])):
                raise ValueError(f"dn CSV must have columns {required}, got {reader.fieldnames}")

            for row in reader:
                if row["split"] != self.split:
                    continue

                image_id = row["image_id"].strip()   # stem (no extension)
                lab = row["label"].strip().lower()

                if lab == "day":
                    dn = 0
                elif lab == "night":
                    dn = 1
                else:
                    # if CSV contains anything else, it's a data error
                    raise ValueError(f"Invalid dn label in CSV for {image_id}: {row['label']}")

                dn_map[image_id] = dn

        self.dn_map = dn_map

        # ---------- FILTER OUT UNLABELED IMAGES ----------
        before = len(self.img_paths)
        self.img_paths = [p for p in self.img_paths if p.stem in self.dn_map]
        after = len(self.img_paths)
        if after == 0:
            raise RuntimeError(f"No labeled (day/night) images found for split='{self.split}'.")
        # --------------------------------------------------

    def __len__(self):
        return len(self.img_paths)

    @staticmethod
    def _read_yolo_labels(label_path: Path) -> torch.Tensor:
        if not label_path.exists():
            return torch.zeros((0, 5), dtype=torch.float32)

        rows = []
        with open(label_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                if len(parts) != 5:
                    continue
                cls, x, y, w, h = parts
                rows.append([float(cls), float(x), float(y), float(w), float(h)])

        if len(rows) == 0:
            return torch.zeros((0, 5), dtype=torch.float32)
        return torch.tensor(rows, dtype=torch.float32)

    @staticmethod
    def _pil_to_chw_float(img: Image.Image) -> torch.Tensor:
        arr = np.array(img, dtype=np.float32) / 255.0
        arr = np.transpose(arr, (2, 0, 1))
        return torch.from_numpy(arr)

    def __getitem__(self, idx: int):
        img_path = self.img_paths[idx]
        stem = img_path.stem

        label_path = self.lbl_dir / f"{stem}.txt"
        mask_path = self.msk_dir / f"{stem}.png"

        if not mask_path.exists():
            raise FileNotFoundError(f"Missing mask for {stem}: {mask_path}")

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        labels = self._read_yolo_labels(label_path)

        if img.size != (self.imgsz, self.imgsz):
            img = img.resize((self.imgsz, self.imgsz), resample=Image.BILINEAR)
        if mask.size != (self.imgsz, self.imgsz):
            mask = mask.resize((self.imgsz, self.imgsz), resample=Image.NEAREST)

        img_t = self._pil_to_chw_float(img)
        mask_np = (np.array(mask, dtype=np.uint8) > 0).astype(np.float32)
        mask_t = torch.from_numpy(mask_np)[None, :, :]

        # dn is ALWAYS valid because we filtered img_paths
        dn = torch.tensor(self.dn_map[stem], dtype=torch.long)

        return img_t, labels, mask_t, dn


In [7]:
import torch
import torch.nn as nn


class ChannelAttention(nn.Module):
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(channels // reduction, 1)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        # shared MLP (implemented with 1x1 convs)
        self.mlp = nn.Sequential(
            nn.Conv2d(channels, hidden, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, channels, kernel_size=1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        w = self.sigmoid(avg_out + max_out)  # BxCx1x1
        return x * w


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size: int = 7):
        super().__init__()
        assert kernel_size in (3, 7)
        padding = (kernel_size - 1) // 2

        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.last_sa = None  # <--- add this

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mean_map = torch.mean(x, dim=1, keepdim=True)
        max_map, _ = torch.max(x, dim=1, keepdim=True)
        m = torch.cat([mean_map, max_map], dim=1)

        w = self.sigmoid(self.conv(m))  # Bx1xHxW in [0,1]
        self.last_sa = w.detach()
        return x * w


class CBAM(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, spatial_kernel: int = 7):
        super().__init__()
        self.ca = ChannelAttention(channels, reduction=reduction)
        self.sa = SpatialAttention(kernel_size=spatial_kernel)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.ca(x)
        x = self.sa(x)
        return x


In [8]:
import torch


x = torch.randn(2, 128, 80, 80)
m = CBAM(channels=128, reduction=16, spatial_kernel=7)

y = m(x)

assert y.shape == x.shape, (x.shape, y.shape)
assert not torch.isnan(y).any(), "NaNs found in CBAM output"
print("CBAM sanity OK:", y.shape)


CBAM sanity OK: torch.Size([2, 128, 80, 80])


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO


class YOLOv8DetSemSeg(nn.Module):
    """
    YOLOv8 detection model + tiny semantic seg head.
    Captures NECK features by hooking the Detect head INPUT (multi-scale features).
    """
    def __init__(self, yolo_weights: str = "yolov8n.pt", use_cbam: bool = False):
        super().__init__()
        self.yolo = YOLO(yolo_weights).model  # nn.Module
        self.use_cbam = use_cbam

        self.cbam_backbone = None  # Point 1 (after last backbone block)
        self.cbam_neck = None      # Point 2 (CBAM on ALL neck feature maps)

        self._neck_feats = None

        self.sem_head = nn.Sequential(
            nn.LazyConv2d(64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=1)
        )

        self.dn_head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),  # [B,C,1,1]
            nn.Flatten(1),            # [B,C]
            nn.LazyLinear(2)          # [B,2] logits: [day, night]
        )

        self._register_backbone_hook_point1()
        self._register_detect_input_hook()

    def _register_backbone_hook_point1(self):
        idx_up = None
        for i, m in enumerate(self.yolo.model):
            if isinstance(m, nn.Upsample) or "upsample" in m.__class__.__name__.lower():
                idx_up = i
                break
        if idx_up is None or idx_up == 0:
            raise RuntimeError("Could not find neck start (Upsample) to place backbone CBAM.")

        backbone_last = self.yolo.model[idx_up - 1]

        if hasattr(self, "_bb_hook_handle") and self._bb_hook_handle is not None:
            self._bb_hook_handle.remove()

        def fwd_hook(module, inputs, output):
            if not self.use_cbam:
                return None
            if self.cbam_backbone is None:
                self.cbam_backbone = CBAM(channels=output.shape[1]).to(output.device)
            return self.cbam_backbone(output)

        self._bb_hook_handle = backbone_last.register_forward_hook(fwd_hook)

    def _register_detect_input_hook(self):
        if not hasattr(self.yolo, "model"):
            raise RuntimeError("Unexpected Ultralytics model: no .model")

        detect_module = self.yolo.model[-1]
        name = detect_module.__class__.__name__.lower()
        if "detect" not in name:
            raise RuntimeError(f"Last module is not Detect (got {detect_module.__class__.__name__}).")

        if hasattr(self, "_detect_hook_handle") and self._detect_hook_handle is not None:
            self._detect_hook_handle.remove()

        def pre_hook(module, inputs):
            feats = list(inputs[0])  # list of multiscale neck features

            if self.use_cbam:
                # one CBAM per scale
                if self.cbam_neck is None:
                    self.cbam_neck = nn.ModuleList([CBAM(f.shape[1]).to(f.device) for f in feats])
                # apply to ALL scales
                feats = [m(f) for m, f in zip(self.cbam_neck, feats)]

            self._neck_feats = feats
            return (feats,) if self.use_cbam else None

        self._detect_hook_handle = detect_module.register_forward_pre_hook(pre_hook)

    @staticmethod
    def _pick_high_res_from_detect_inputs(feats):
        if not isinstance(feats, (list, tuple)) or len(feats) == 0:
            raise RuntimeError("Detect input features not captured.")
        return max(feats, key=lambda t: t.shape[-2] * t.shape[-1])

    @staticmethod
    def _pick_low_res_from_detect_inputs(feats):
        if not isinstance(feats, (list, tuple)) or len(feats) == 0:
            raise RuntimeError("Detect input features not captured.")
        return min(feats, key=lambda t: t.shape[-2] * t.shape[-1])

    def forward(self, x):
        # TRAIN: x is a batch dict -> YOLO returns (det_loss, loss_items)
        if isinstance(x, dict):
            self._neck_feats = None
            imgs = x["img"]
            det_loss, det_items = self.yolo(x)

            feat_seg = self._pick_high_res_from_detect_inputs(self._neck_feats)
            seg_logits = self.sem_head(feat_seg)
            seg_logits = F.interpolate(seg_logits, size=imgs.shape[-2:], mode="bilinear", align_corners=False)

            feat_dn = self._pick_low_res_from_detect_inputs(self._neck_feats)
            dn_logits = self.dn_head(feat_dn)

            return det_loss, det_items, seg_logits, dn_logits

        # INFER: x is an image tensor -> YOLO returns preds
        self._neck_feats = None
        det_preds = self.yolo(x)

        feat_seg = self._pick_high_res_from_detect_inputs(self._neck_feats)
        seg_logits = self.sem_head(feat_seg)
        seg_logits = F.interpolate(seg_logits, size=x.shape[-2:], mode="bilinear", align_corners=False)

        feat_dn = self._pick_low_res_from_detect_inputs(self._neck_feats)
        dn_logits = self.dn_head(feat_dn)

        return det_preds, seg_logits, dn_logits


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [10]:
import torch

def collate_det_seg(batch):
    # batch items: (img, labels, mask, dn)
    imgs, labels_list, masks, dns = zip(*batch)

    imgs = torch.stack(imgs, 0)         # [B,3,H,W]
    masks = torch.stack(masks, 0)       # [B,1,H,W]
    dn = torch.tensor(dns, dtype=torch.long)  # [B]

    bboxes_all, cls_all, batch_idx_all = [], [], []
    for i, lab in enumerate(labels_list):
        if lab.numel() == 0:
            continue
        cls = lab[:, 0:1].long()
        bboxes = lab[:, 1:5].float()
        bboxes_all.append(bboxes)
        cls_all.append(cls)
        batch_idx_all.append(torch.full((lab.shape[0],), i, dtype=torch.long))

    if len(bboxes_all):
        bboxes = torch.cat(bboxes_all, 0)
        cls = torch.cat(cls_all, 0)
        batch_idx = torch.cat(batch_idx_all, 0)
    else:
        bboxes = torch.zeros((0, 4), dtype=torch.float32)
        cls = torch.zeros((0, 1), dtype=torch.long)
        batch_idx = torch.zeros((0,), dtype=torch.long)

    # IMPORTANT: YOLO batch dict must contain ONLY what YOLO needs
    yolo_batch = {"img": imgs, "bboxes": bboxes, "cls": cls, "batch_idx": batch_idx}

    # return extras separately (so YOLO loss doesn't see them)
    return yolo_batch, masks, dn



In [11]:
import torch

@torch.no_grad()
def val_iou(model, loader, device, max_batches=None):
    model.eval()
    total_iou = 0.0
    total_imgs = 0

    for bi, (det_batch, mask, dn) in enumerate(loader):
        if (max_batches is not None) and (bi >= max_batches):
            break

        det_batch = {k: v.to(device, non_blocking=True) for k, v in det_batch.items()}
        gt = (mask.to(device, non_blocking=True) > 0.5).float()

        det_loss, det_items, seg_logits, dn_logits = model(det_batch)  # dict-path returns 4
        pred = (torch.sigmoid(seg_logits) > 0.5).float()

        inter = (pred * gt).sum(dim=(1, 2, 3))
        union = ((pred + gt) > 0).float().sum(dim=(1, 2, 3)).clamp_min(1.0)

        iou = inter / union
        total_iou += iou.sum().item()
        total_imgs += iou.numel()

    return total_iou / max(1, total_imgs)


In [14]:
test_ds = BDDDetDrivableDataset(
    yolo_root="/content/BDD100K_640/yolo_640",
    mask_root="/content/BDD100K_640/drivable_masks_640",
    split="test",
    imgsz=640,
    dn_csv_path="/content/daynight_labels.csv"
)

from torch.utils.data import DataLoader
import torch

test_loader = DataLoader(
    test_ds,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available(),
    persistent_workers=False,
    collate_fn=collate_det_seg
)


In [21]:

# FINAL TRI-TASK EVALUATION
#  - Detection: Ultralytics .val() on val + test
#  - Segmentation: mean IoU on val + test (binary)
#  - Day/Night: acc + balanced acc + F1 on val + test


import os
import torch
import torch.nn.functional as F
from ultralytics import YOLO
from ultralytics.models.yolo.detect.train import DetectionTrainer

device = "cuda" if torch.cuda.is_available() else "cpu"
dev_override = 0 if device == "cuda" else "cpu"

# PATHS
data_yaml = "/content/BDD100K_640/yolo_640/dataset_640.yaml"

# tri task cbam weights
best_wrapper_pt = "/content/drive/MyDrive/XAI_Project/experiments/det_seg_dn_cbam_640/weights/best.pt"

# a YOLO checkpoint that Ultralytics can load
det_base_pt = "/content/drive/MyDrive/XAI_Project/experiments/det_baseline/weights/best.pt"

imgsz = 640
batch = 8


# 1) Building the model

model = YOLOv8DetSemSeg(yolo_weights=det_base_pt, use_cbam=True).to(device)

trainer = DetectionTrainer(overrides={
    "model": det_base_pt,
    "data":  data_yaml,
    "imgsz": imgsz,
    "device": dev_override,
    "batch": batch,
})
trainer.setup_model()
trainer.model.args = trainer.args
trainer.model.init_criterion()

# swap YOLO model into wrapper
model.yolo = trainer.model.to(device)

# re-hook
model._neck_feats = None
model.cbam_backbone = None
model.cbam_neck = None
model._register_backbone_hook_point1()
model._register_detect_input_hook()

# load wrapper state_dict
sd = torch.load(best_wrapper_pt, map_location=device)
if isinstance(sd, dict) and "state_dict" in sd:
    sd = sd["state_dict"]
model.load_state_dict(sd, strict=False)
model.eval()

print("OK: loaded wrapper best:", best_wrapper_pt)


# 2) DETECTION EVAL

yolo = YOLO(det_base_pt)
yolo.model = model.yolo  # use the swapped, trained model
yolo.model.names = {0:"car", 1:"truck", 2:"bus", 3:"person", 4:"bike"}


print("\n=== DETECTION: TEST ===")
try:
    det_test = yolo.val(data=data_yaml, imgsz=imgsz, device=dev_override, split="test", batch=batch)
except Exception as e:
    det_test = None
    print("No test split or error running test val():", repr(e))


# 3) SEG + DAY/NIGHT eval

@torch.no_grad()
def eval_seg_dn(model, loader, device, max_batches=None):
    model.eval()

    # seg IoU
    inter_sum = 0.0
    union_sum = 0.0

    # dn metrics (ignore dn=-1)
    tp = tn = fp = fn = 0

    n_batches = 0
    for bi, (det_batch, mask, dn) in enumerate(loader):
        if (max_batches is not None) and (bi >= max_batches):
            break

        det_batch = {k: v.to(device, non_blocking=True) for k, v in det_batch.items()}
        gt = (mask.to(device, non_blocking=True) > 0.5).float()
        dn = dn.to(device, non_blocking=True).long()
        _, _, seg_logits, dn_logits = model(det_batch)


        pred = (torch.sigmoid(seg_logits) > 0.5).float()

        inter = (pred * gt).sum(dim=(1,2,3))
        union = ((pred + gt) > 0).float().sum(dim=(1,2,3))
        inter_sum += inter.sum().item()
        union_sum += union.sum().item()

        valid = (dn >= 0)
        if valid.any():
            pred_dn = torch.argmax(dn_logits[valid], dim=1)
            true_dn = dn[valid]

            # positive class = 1 (night), negative = 0 (day)
            tp += int(((pred_dn == 1) & (true_dn == 1)).sum().item())
            tn += int(((pred_dn == 0) & (true_dn == 0)).sum().item())
            fp += int(((pred_dn == 1) & (true_dn == 0)).sum().item())
            fn += int(((pred_dn == 0) & (true_dn == 1)).sum().item())

        n_batches += 1

    miou = (inter_sum / max(1.0, union_sum))

    # dn acc/balanced acc/f1
    total = tp + tn + fp + fn
    acc = (tp + tn) / max(1, total)

    tpr = tp / max(1, (tp + fn))  # recall night
    tnr = tn / max(1, (tn + fp))  # recall day
    bal_acc = 0.5 * (tpr + tnr)

    prec = tp / max(1, (tp + fp))
    rec  = tpr
    f1 = (2 * prec * rec) / max(1e-12, (prec + rec))

    cm = {"tn": tn, "fp": fp, "fn": fn, "tp": tp}
    return miou, acc, bal_acc, f1, cm


print("\n=== SEG + DAY/NIGHT: TEST  ===")
try:
    test_miou, test_acc, test_bal, test_f1, test_cm = eval_seg_dn(model, test_loader, device)
    print(f"SEG mean IoU: {test_miou:.4f}")
    print(f"DN acc: {test_acc:.4f} | balanced acc: {test_bal:.4f} | F1(night=1): {test_f1:.4f}")
    print("DN confusion (tn,fp,fn,tp):", test_cm)
except NameError:
    print("No test_loader defined. (If you have a test split, build test_ds/test_loader like val_loader.)")




Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/BDD100K_640/yolo_640/dataset_640.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/XAI_Project/experiments/det_baseline/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train4, nbs=64, nms=False, opset=None, optimize=Fal